In [ ]:
from elasticsearch import helpers, Elasticsearch
import csv
import pandas as pd
pd.options.display.max_columns = None
from elasticsearch import Elasticsearch
from simpledbf import Dbf5

# Для начала преобразуем всё в csv

In [ ]:
files = glob.glob('FIAS/ADDR*', recursive=True)
for i, f in enumerate(files):
  if f[-3:].lower() == 'dbf':
    print('processing {0} of {1}. Filename: {2}           '.format(i+1, len(files), f), end='\r')
    dbf = Dbf5(f , codec='cp866')
    dbf.to_csv('fias_csv/ADDROBJ.csv')

In [ ]:
path = 'fias/'
files = ['ESTSTAT.DBF', 'FLATTYPE.DBF', 'HSTSTAT.DBF', 'INTVSTAT.DBF', 'NDOCTYPE.DBF', 
  'OPERSTAT.DBF', 'ROOMTYPE.DBF', 'SOCRBASE.DBF', 'STRSTAT.DBF']
for i, f in enumerate(files):
  if f[-3:].lower() == 'dbf':
    print('processing {0} of {1}. Filename: {2}           '.format(i+1, len(files), f), end='\r')
    dbf = Dbf5(path+f , codec='cp866')
    dbf.to_csv('fias_csv/{0}.csv'.format(f[:-4]))

In [ ]:
files = glob.glob('FIAS/HOUSE*', recursive=True)
for i, f in enumerate(files):
  if f[-3:].lower() == 'dbf':
    print('processing {0} of {1}. Filename: {2}           '.format(i+1, len(files), f), end='\r')
    dbf = Dbf5(f , codec='cp866')
    dbf.to_csv('fias_csv/HOUSE.csv')

In [ ]:
files = glob.glob('FIAS/ROOM*', recursive=True)
for i, f in enumerate(files):
  if f[-3:].lower() == 'dbf':
    print('processing {0} of {1}. Filename: {2}           '.format(i+1, len(files), f), end='\r')
    dbf = Dbf5(f , codec='cp866')
    dbf.to_csv('fias_csv/ROOM.csv')

In [ ]:
files = glob.glob('FIAS/STEAD*', recursive=True)
for i, f in enumerate(files):
  if f[-3:].lower() == 'dbf':
    print('processing {0} of {1}. Filename: {2}           '.format(i+1, len(files), f), end='\r')
    dbf = Dbf5(f , codec='cp866')
    dbf.to_csv('fias_csv/STEAD.csv')

# Теперь надо всё это закинуть в Elastic
Да, это не самый оптимальный путь (можно миновать csv). Но это уже как есть

In [ ]:
es = Elasticsearch()

In [ ]:
def load_elastic(fn, index, doc_type, encoding='cp866', es=es):
  '''
  Этот метод загружает указанный файлик в elastic.
  '''
  with open(fn, encoding=encoding) as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index=index, doc_type=doc_type, raise_on_error=False, stats_only=True)
    print('done')

In [ ]:
# Загрузка самой главной таблицы
# На первых порах её нам хватит. Остальные загружаются при надобности
# Занимает 2 часа
load_elastic('fias_csv/ADDROBJ.csv', 'fias', 'address')

# Оптимизация под полнотекстовый поиск

In [ ]:
def full_address(GUID):
  answer = es.search(index='fias', doc_type='address', body=
  {
    "size":1,
    "query": {
      "bool": {
        "must": [
          { "match": { 
              "AOGUID": GUID }},
          { "match": { 
              "ACTSTATUS":  1}}
        ]
      }
    }
  })
  entry = answer["hits"]["hits"][0]["_source"]
  string = entry['SHORTNAME'] + " " + entry['OFFNAME']
  if len(entry['PARENTGUID'])>5:
    string = full_address(entry['PARENTGUID']) + ', ' + string
  return string

In [ ]:
df_addr = pd.read_csv('fias_csv/ADDROBJ.csv', encoding='cp866', dtype=str, error_bad_lines=False)

In [ ]:
def full_address_sep(GUID, _leaf=True):
  address = {}
  address['fullname'] = ''    
  answer = es.search(index='fias', doc_type='address', body=
  {
    "size":1,
    "query": {
      "bool": {
        "must": [
          { "match": { 
              "AOGUID": GUID }},
          { "match": { 
              "ACTSTATUS":  1}}
        ]
      }
    }
  })
  entry = answer["hits"]["hits"][0]["_source"]
  level = entry["AOLEVEL"]
  LUT = {
    '1':'region',
    '3':'area',
    '4':'city',
    '5':'district',
    '6':'town',
    '7':'street', 
    '90':'additional',
    '91':'nestreet'
  }
  if _leaf:
    address['guid'] = GUID
    address['aolevel'] = entry['AOLEVEL']
  address[LUT.get(level, level)] = entry['OFFNAME']
  address[LUT.get(level, level)+"_type"] = entry['SHORTNAME']
  address['fullname'] = entry['SHORTNAME'] + " " + entry['OFFNAME']
  if len(entry['PARENTGUID'])>5:
    nest = full_address_sep(entry['PARENTGUID'], _leaf=False)
    string = nest['fullname'] + ', ' + address['fullname']
    address.update(nest)
    address['fullname'] = string
  return address

In [ ]:
# здесь могут быть ошибки парсинга на некоторых полях. 
# Их можно просто пропустить а потом попытаться исправить самостоятельно.
start= None
finish = None
i = start
for _, value in df_addr[["AOGUID"]][df_addr['ACTSTATUS']=='1'][start:finish].iterrows():
  if i%50==0:
    print(i,  end="\r")
  full_addr = full_address_sep(value["AOGUID"])
  es.index(index="fias_full_text", id=value["AOGUID"], doc_type='address', body=full_addr)
  i+=1

На данном этапе в elastic должна быть таблица fias_full_text. Далее мы её будем максимально активно использовать

In [ ]:
# Можно ставить на ночь. Это очень долго: 18Гб таблица весит
load_elastic('fias_csv/HOUSE.csv', 'fias_houses', 'home')